# Objetivo: Previsão de pacientes na UTI (COVID-19)

Utilizarei a base de dados do Hospital Sírio Libanês. A base de dados esta disponível no site do [Kaggle](https://www.kaggle.com/) na pagina do grupo do Sírio Libanês [COVID-19 - Clinical Data to assess diagnosis](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19).

Essa base de dados contém informações, não sensíveis, que diz respeito a quantidade de pacientes que foram ou não internados por covid-19 na clínica do hospital durante a pandemia de corona virus. As informações são ricas com respeito ao quadro clínico e a pergunta que vamos tentar responder é: 

> **Dado um novo paciente conseguiremos prever a chance dele ser encaminhado para a UTI?**


In [1]:
import pandas as pd

In [2]:
sirio_libanes = pd.read_excel('https://github.com/ConradBitt/BootCamp_DataScience/blob/master/ML%20em%20Saude/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true')
sirio_libanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Columns: 231 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: float64(225), int64(4), object(2)
memory usage: 3.4+ MB


In [3]:
sirio_libanes.sample(5).T

,1275,1241,482,409,39
PATIENT_VISIT_IDENTIFIER,255,248,96,81,7
AGE_ABOVE65,1,0,1,1,0
AGE_PERCENTIL,80th,50th,80th,70th,20th
GENDER,0,1,0,0,0
DISEASE GROUPING 1,0,0,1,0,0
...,...,...,...,...,...
RESPIRATORY_RATE_DIFF_REL,NaN,NaN,-1,0.422842,-0.817204
TEMPERATURE_DIFF_REL,NaN,NaN,-1,-0.423796,-0.690476
OXYGEN_SATURATION_DIFF_REL,NaN,NaN,-1,-0.899078,-0.920103
WINDOW,0-2,2-4,4-6,ABOVE_12,ABOVE_12


## Tarefas impostas pelo Kaggle 

> **Task 1**: Preve a admissão na UTI os casos confirmados de covid-19. Com base na amostra acima, verificar a viabilidade de previsão/classificação dos pacientes que precisarão ou não do suporte de terapia intensiva do hospital. O objetivo é fornecer ao hospital uma previsão de quatro ou três semenas de antecedência da forma mais acurada possível para que os recursos utilizados na UTI possam ser arranjados ou remanejados. 

> **Task 2**: Prever a **NÃO** admissão dos casos de covid-19 na UTI. Com base na amostra dos dados, prever quais pacientes não preciarão de suporte a unidade de terapia intensiva. O objetivo é fornecer aos hospitais temporários e locais uma resposta boa o suficiente para que os médicos de linha de frente possam dar alta com segurança e acompanhar pacientes remotamente.

Note que o objetivo não se reduz a fazer uma análise de estatistica descritiva, o objetivo é classificação de pacientes que precisarão de uma UTI ou não. Ou seja, **uma classificação binária.**

---

## Vamos verificar qual a quantidade dos dados que foram ou não pra UTI:

In [4]:
sirio_libanes['ICU'].value_counts(normalize=True).round(2)*100

0    73.0
1    27.0
Name: ICU, dtype: float64

### Comentário sobre a proporção de pacientes que precisaram da UTI
> Acima podemos verificar que nesta base de dados, dentre os 1924 registros, a quantidade de pessoas que evoluem até a situação em que necessitam de uma UTI é de quase 27%, os outros 73% precisaram do serviço.

## A respeito da anonimização dos dados

A Lei Geral de Proteção de Dados Pessoais (LGPD), Lei nº 13.709, de 14 de agosto de 2018, dispõe sobre o tratamento de dados pessoais, inclusive nos meios digitais, por pessoa natural ou por pessoa jurídica de direito público ou privado, com o objetivo de proteger os direitos fundamentais de liberdade e de privacidade e o livre desenvolvimento da personalidade da pessoa natural. 

Tendo em vista a vigência dessa Lei, de suma importancia pra sociedade, é importante que os dados sejam anonimizados para respeitar os direitos dos cidadãos brasileiros. É importante citar isso pois anonimização de um dado deve ser muito bem feita afim de não violar o direito dos individuos.

Note que só o fato de sabermos que a mostra foi retirada dentre os pacientes do Hospital Sírio Libanês em um intervalo X de tempo já reduz muito o fator de anonimidade do dado e se uma variável fosse explicitamente a comorbidade "diabetes", "pressão alta" ou "HIV", caso as variáveis não fossem *clusterizadas*, sabendo a prevalência de uma doença numa população é possível encontrar as características de cada amostra e é importante dificultar este tipo de análise para estar de acordo com a lei de proteção de dados.

Uma forma de anonimizar esses dados é criar grupos de características entre as amostras evitando assim de informar exatamente quais são as características de um elemento da base de dados. 

fonte: [Lei Geral de Proteção de Dados - LGPD](https://www.planalto.gov.br/ccivil_03/_Ato2015-2018/2018/Lei/L13709.htm#ementa)

## Propondo modelo Linear

Um modelo matemático pode ser expresso em uma função, tal que: 

### $$\vec{y} = \vec{f}(\vec{x})$$

no caso $\vec{f}$  é um operador vetorial que irá realizar alguma transformação nos dados $\vec{x}$ resultando em $\vec{y}$. Note que em geral temos $\vec{x}$ e $\vec{y}$ e queremos ajustar um modelo $\vec{f}$.

> O vetor $\vec{y}$ é chamado de *variável dependente* pois ele é obtido através de uma função do vetor $\vec{x}$. O vetor $\vec{x}$ é dito variável independente, pois são dados que o estatistico/médico/pesquisador acredita que possam contribuir para entender o valor de $\vec{y}$.

### Dados de treino e dados de teste

Técnicas de regressão em geral tentam ajustar uma curva a um conjunto de dados e posteriormente após ter a curva parte da análise é entender como variam os **resíduos** que no caso é a diferença entre um dado e o resultado predito pelo modelo. Em aprendizado de máquina esta técnica também é utilizada porém com uma modificação.

Os dados que serão fornecidos pelo modelo são classificados em **dados de treino** e **dados de teste**. Aos dados de treino a técnica é exatamente a mesma usada em técnica de regressões: 

> Usa $x$ **entradas de treino** pra ajustar um modelo $f$ que vai calcular uma **saida** $y'$ prevista pelo modelo. Então compara-se $y$ real com $y'$ previsto.

A introdução do conceito dos **dados de teste** é garantir que mesmo que com $x$ dados de entrada nunca vistos o modelo $f$ consegue ainda assim abstrair novas informações nunca vistas afim de estimar $y'$ cujo o resultado é o melhor valor de $y$.

> Usa $x$ entradas de treino pra ajustar um modelo $f$ que vai calcular uma saida $y'$ prevista pelo modelo. Então compara-se $y$ real com $y'$ previsto. Agora afim de testar a abstração do modelo, fornecemos $X$ **entradas de teste** para verificar a precisão de $f$ ao gerar $Y'$ **saidas previstas** em relação aos $Y$ **saidas de teste**

Um ponto importante é que a abstração do modelo esta sempre quantificada em relação aos dados de teste, não aos dados de treino. Isso porque os dados de treino são usados para treinar e estimar um modelo, já os dados de teste são dados que o modelo não teve contato e portanto as informações do $X_{teste}$ são novas ao modelo então precisamos verificar o quão bom o modelo é pra para estimar resultados em cima de dados que ele nunca viu antes. Uma forma bem simples de entender esse algorítmo de treinamento e teste é: 

1. Separa a base de dados em dados de treino e dados de teste.
2. Usa os dados de treino ($x,y$) para ajustar $f$.
3. Usa o modelo $f$ ajustado em dados de teste.
4. Comparar sempre o resultado em relação aos dados de teste.

In [5]:
colunas_quantitativas = sirio_libanes.describe().columns

sirio_libanes_quantitativos = sirio_libanes[colunas_quantitativas].dropna()
sirio_libanes_quantitativos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 545 entries, 4 to 1924
Columns: 229 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: float64(225), int64(4)
memory usage: 979.3 KB


In [6]:
y = sirio_libanes_quantitativos['ICU']
x = sirio_libanes_quantitativos.drop('ICU', axis=1)

In [7]:
display(y.shape)
display(x.shape)

(545,)

(545, 228)

### Modelo de Regressão Logística

A regressão logística é uma técnica estatística que tem como objetivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, a partir de uma série de variáveis explicativas contínuas e/ou binárias.

* Em medicina, permite por exemplo determinar os factores que caracterizam um grupo de indivíduos doentes em relação a indivíduos sãos.
* No domínio dos seguros, permite encontrar fracções da clientela que sejam sensíveis a determinada política securitária em relação a um dado risco particular.
* Em instituições financeiras, pode detectar os grupos de risco para a subscrição de um crédito.
* Em econometria, permite explicar uma variável discreta, como por exemplo as intenções de voto em actos eleitorais.

#### Considerações do modelo: 
* Relação linear entre o vetor das variáveis explicativas X e o logit da variável resposta Y
* Ausência de multicolinearidade
* Valor esperado dos resíduos igual a zero
* Ausência de heterocedasticidade
* Não pressupõe normalidade dos resíduos nem homogeneidade de variâncias. 

#### SK Learn
```python
sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
```
[[source]](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
modelo_regressao_logistica = LogisticRegression()

In [10]:
modelo_regressao_logistica.fit(x,y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Com o modelo treinado (*fit*) podemos usa-lo para predizer algo:

In [11]:
modelo_regressao_logistica.predict([x.iloc[5]])

array([0])

Note que para o dado `[x.iloc[5]]` o modelo sugere um resultado `array([0])` ou seja, que este paciente não vai para a UTI. Podemos verificar esta situação nos dados de saida:

In [12]:
y.iloc[5]

0

realmente o resultado condiz com a previsão do modelo. Entretanto ainda existe outros dois problemas fundamentais. Primeiro problema é: não adianta acertar **um resultado**, é preciso mensurar a taxa de acerto. O segundo é a **separação entre os dados de treino e teste**, não adianta prever dados que foram usados para ensinar o modelo. 

#### Para resolver o Primeiro problema

Em vez de prever um único elemento podemos fornecer vários dados ao mesmo tempo:

In [13]:
y_previsto = modelo_regressao_logistica.predict(x)

agora basta comparar `y_previsto` com `y`

In [14]:
y_previsto == y

4        True
6        True
8        True
9        True
14       True
        ...  
1904    False
1914     True
1919     True
1921     True
1924     True
Name: ICU, Length: 545, dtype: bool

Note que ele retorna um array booleano. Como no python `True == 1` e `False == 0 `

In [15]:
True == 1 

True

podemos somar todos os valores do array booleano e verificar quantos foram os acertos:

In [16]:
sum(y_previsto == y)

497

In [17]:
sum(y_previsto == y) / len(y) * 100

91.19266055045871

#### Conclusão a respeito do primeiro problema
> Existe uma regra de ouro **"Se você treinou um modelo e a acurrácia foi de 90% logo na primeira simulação, então provávelmente tem algum engano."** 

> O resultado foi de 497 acertos, uma porporção de 91% dos dados. Esta é uma forma de metrificar a qualidade do modelo, acontece que ainda temos o problema de tentar prever dados que o modelo já utilizou na faze de treino, não é um bom indicador de qualidade testar um modelo em cima dos dados que foram utilizados para treina-lo.


### Baseline com Dummy Classifier 

Além disso, existe a possibilidade do modelo conseguir acertar por mera chance, por exemplo, temos 545 dados, com valor $0$ ou $1$, se o modelo atribuir $1$ para todos, já irá acertar 264 pacientes... Não por previsão mas por mera chance. Existe uma forma de medir essa "mera chance" com um classificador "ingênuo" ou ["dummy classifier"](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.dummy)


```python
sklearn.dummy.DummyClassifier(*, strategy='prior', random_state=None, constant=None)[source]
```


In [18]:
from sklearn.dummy import DummyClassifier

modelo_dummy = DummyClassifier()
modelo_dummy.fit(x, y)
sum(modelo_dummy.predict(x) == y) / len(y) * 100

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


49.72477064220184

#### Conclusão sobre dummy classifier:
> Ou seja, com o ``DummyClassifier`` por mera chance acerta $49,35\%$ dos dados. Mas não é só ele, se você selecionar uma pessoa qualquer, aleatória, a chance dela acertar se um paciente vai ou não pra UTI é $49,35\%$.

> Note que a estrategia de decisão classifier esta na documentaçãão dizendo **“prior”: always predicts the class that maximizes the class prior (like “most_frequent”) and predict_proba returns the class prior.**  Ou seja, ele analisa a proporção, se tem $51\%$ de zeros, ele vai chutar $51\%$ zeros e vai acertar na maioria das vezes...

>  Então o objetivo do nosso modelo é informar um resultado que não seja por mera chance. 

### Metrificando os modelos com [``Accuracy Score``](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html?highlight=accuracy%20score#sklearn.metrics.accuracy_score)

O método ``accuracy_score()`` é um jeito mais automatizado de calcular acurácia de um modelo:

```python
sklearn.metrics.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
```

Accuracy classification score.

In multilabel classification, this function computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in `y_true`.



In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_previsto)

0.9119266055045872

### Separando os dados de treino e teste

Existe uma função dentro do SK Learn que faz a "quebra" dos dados em conjuntos de treino e teste:

```python
sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
```

> por definição o método separa $25\%$ dos dados para o conjunto de teste, mas isso pode ser modificado nos hiperparâmetros da função.

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y)

print('Dados de Treino: ',len(x_treino),len(y_treino),
      '\nDados de Teste:',len(x_teste), len(y_teste))

Dados de Treino:  408 408 
Dados de Teste: 137 137


### Treinando modelo logistico com dados de treino

Agora podemos usar os dados de treino para treinar o modelo:

In [22]:
modelo_regressao_logistica.fit(x_treino,y_treino)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
y_predito = modelo_regressao_logistica.predict(x_teste)

In [24]:
accuracy_score(y_teste, y_predito)

0.8613138686131386

### Semente e o ``Random_state``

Um ponto importante é que o método ``train_test_split()`` faz uma seleção aleatória dos dados de treino e teste, portanto a cada execução a metrica de qualidade de um modelo muda, veja:

In [25]:
epocas = {}
for epoca in range(10):
  # Instanciando modelos
  modelo_dummy = DummyClassifier()
  modelo_regressao_logistica = LogisticRegression(solver='newton-cg')

  # Separação dos dados
  x_treino, x_teste, y_treino, y_teste = train_test_split(x,y)

  # Ajuste dos modelos
  modelo_dummy.fit(x_treino,y_treino)
  modelo_regressao_logistica.fit(x_treino,y_treino)

  # Predição dos modelos
  y_pred_dummy = modelo_dummy.predict(x_teste.to_numpy())
  y_pred_logis = modelo_regressao_logistica.predict(x_teste.to_numpy())

  # Métricas de qualidade
  acc_dummy = accuracy_score(y_teste, y_pred_dummy)
  acc_logis = accuracy_score(y_teste, y_pred_logis)

  # Exibindo méétricas
  epocas[epoca] = (acc_dummy, acc_logis)


for chave, valor in epocas.items():
  print(f'Epoca {chave} - '+'\033[34m'+f'Acurácia Dummy: {valor[0].round(2)*100:.2f}% | Acurácia Logistic: {valor[1].round(2)*100:.2f}% '+'\033[0;0m')

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarn

Epoca 0 - Acurácia Dummy: 47.00% | Acurácia Logistic: 80.00% 
Epoca 1 - Acurácia Dummy: 47.00% | Acurácia Logistic: 90.00% 
Epoca 2 - Acurácia Dummy: 54.00% | Acurácia Logistic: 85.00% 
Epoca 3 - Acurácia Dummy: 50.00% | Acurácia Logistic: 88.00% 
Epoca 4 - Acurácia Dummy: 52.00% | Acurácia Logistic: 89.00% 
Epoca 5 - Acurácia Dummy: 45.00% | Acurácia Logistic: 85.00% 
Epoca 6 - Acurácia Dummy: 48.00% | Acurácia Logistic: 90.00% 
Epoca 7 - Acurácia Dummy: 48.00% | Acurácia Logistic: 91.00% 
Epoca 8 - Acurácia Dummy: 49.00% | Acurácia Logistic: 88.00% 
Epoca 9 - Acurácia Dummy: 42.00% | Acurácia Logistic: 85.00% 


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


#### Comentario sobre variação da precisão 

> Vamos chamar de **Época** um ciclo de separação dos dados, treino e teste. Note na saíída em azul no terminal que para cada época a acurácia do dummy e do logistic muda. Isso acontece porque a forma em que o ``train_test_split()`` seleciona os dados é aleatória, logo cada execução é selecionado um conjunto de dados diferentes e portanto a precisão do modelo varia...

#### Núúmeros aleatóórios

Para garantir que o modelo selecione sempre o mesmo conjunto de dados podemos definir uma "Semente" ou `seed`, que é basicamente um número que origina toda uma sequência de números aleatórios. Com o `numpy` podemos definir uma semente e então fornecemos ela como hiperparâmetro do `train_test_split(random_state=seed)` 

> **Detalhe**: Em muitos códigos por ai existem pessoas que escolhem uma semente específica. Por exemplo, escolher o número $0$, o dia do aniversário ou até mesmo o número $42$, em homenagem ao [Guia do Mochileiro das Galáxias](https://www.amazon.com.br/guia-mochileiro-das-gal%C3%A1xias/dp/8599296574). **Entretanto vale ressaltar que esse número não é aleatório, pois a chance dele ser escolhido entre varias pessoas é bem maior do que o qualquer outro número devido a influencia na cultura pop...**

**Uma sugestão para escolher um número cujo valor irá servir de semente é literalmente digitar varios números com os dedos sem pensar muito (*hahaha!*)**

In [26]:
import numpy as np

seed = np.random.seed(120925)

epocas1 = {}
for epoca in range(1,11):
  # Instanciando modelos
  modelo_dummy = DummyClassifier()
  modelo_regressao_logistica = LogisticRegression(solver='newton-cg')

  # Separação dos dados
  x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, random_state=seed)

  # Ajuste dos modelos
  modelo_dummy.fit(x_treino,y_treino)
  modelo_regressao_logistica.fit(x_treino,y_treino)

  # Predição dos modelos
  y_pred_dummy = modelo_dummy.predict(x_teste.to_numpy())
  y_pred_logis = modelo_regressao_logistica.predict(x_teste.to_numpy())

  # Métricas de qualidade
  acc_dummy = accuracy_score(y_teste, y_pred_dummy)
  acc_logis = accuracy_score(y_teste, y_pred_logis)

  # Exibindo méétricas
  epocas1[epoca] = (acc_dummy, acc_logis)

epoca1 = pd.DataFrame(epocas1, index=['Dummy','Logistic']).T.describe()

for chave, valor in epocas.items():
  print(f'Epoca {chave} - '+'\033[34m'+f'Acurácia Dummy: {valor[0].round(2)*100:.2f}% | Acurácia Logistic: {valor[1].round(2)*100:.2f}% '+'\033[0;0m')

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarn

Epoca 0 - Acurácia Dummy: 47.00% | Acurácia Logistic: 80.00% 
Epoca 1 - Acurácia Dummy: 47.00% | Acurácia Logistic: 90.00% 
Epoca 2 - Acurácia Dummy: 54.00% | Acurácia Logistic: 85.00% 
Epoca 3 - Acurácia Dummy: 50.00% | Acurácia Logistic: 88.00% 
Epoca 4 - Acurácia Dummy: 52.00% | Acurácia Logistic: 89.00% 
Epoca 5 - Acurácia Dummy: 45.00% | Acurácia Logistic: 85.00% 
Epoca 6 - Acurácia Dummy: 48.00% | Acurácia Logistic: 90.00% 
Epoca 7 - Acurácia Dummy: 48.00% | Acurácia Logistic: 91.00% 
Epoca 8 - Acurácia Dummy: 49.00% | Acurácia Logistic: 88.00% 
Epoca 9 - Acurácia Dummy: 42.00% | Acurácia Logistic: 85.00% 


Agora toda vez que a célula for executada, o resultado de cada época será o mesmo, mas a seleção que ``train_test_split()`` não é a mesma, mas caso seja necessário  selecionar sempre o mesmo conjunto de dados éé sóó fixar um valor em ``random_state``.

#### Importancia de uma seleção representativa para treino e teste

Como a seleção feita pelo ``train_test_split()`` é aleatória então pode acontecer alguns problemas ao selecionar um conjunto de dados representativo. Vimos que a proporção dos pacientes que foram ou não pra UTI é de $52\%$ não precisa de UTI e $48\%$ precisa.

Pode acontecer uma situação em que nos dados de treino $90\%$ dos pacientes não precisam de UTI, ou seja, este conjunto de dados não é representativo do conjunto total de dados. Como também pode acontecer em uma seleção que $90\%$ dos pacientes precisem de UTI, logo o modelo vai ser ajustado em um conjunto de dados não representativo. **Isso pode ser bom em alguns casos mas pode não ser.**

A importancia de que os dados de treino e teste sejam representativos é garantir que o modelo consiga treinar corretamente e também abstrair informações para isso podemos passar o hiperparâmetro ``stratify=y``. Assim o modelo irá garantir que todas as amostras estratificadas sejam representativas em relação aos dados de saída real.

Entretanto em outros casos pode ser interessante tentar prever corretamente resultados de amostras estratificadas não representativas. Em outras palavras, ter uma boa taxa de acerto até em dados de teste não representativos. Por exemplo:

> A evolução do COVID acontece num janela em média de 15 dias. Portanto, 15 dias após o carnaval é possíível que os dias tenham um aumento expressivo no número de casos e internações, ou seja, num periodo de tempo a quantidade de dados não será representativa em relação aos dados que foram utilizados para treinar o modelo, pois a taxa de contaminação mudou então a quantidade de pessoas que dão entrada no hospital também muda.

É importante falar disso pois se treinarmos exclusivamente em dados representativos, ao entrar em uma situação de teste cuja a realidade não é mais representativa do conjunto de treino o modelo pode acabar errando muito mais, e neste caso, erar significa **classificar um paciente para a UTI sem necessidade ou não mandar um paciente, mesmo ele precisando.** 

> Este tipo de análise condicional, por exemplo "*Quantos pacientes foram encaminhados à UTI, dado que eles não precisavam*" ou "*Quantos pacientes não foram enviados para a UTI dado que eles precisavam da assistencia*", é referente a diferença fundamental entre **Precisão e Revocação**

Uma imagem que ilustra bem a diferença entre precisão e revocação é

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Precis%C3%A3o_e_revoca%C3%A7%C3%A3o.png/262px-Precis%C3%A3o_e_revoca%C3%A7%C3%A3o.png' >

> No ambiente hospitalar é de suma importância não só ter uma precisão alta, mas uma também uma revocação. Pois, classificar um paciente que precisa de UTI como uma pessoa que não precisa do serviço pode custar a vida do paciente.


Por exemplo, Vamos ver o modelo:

In [27]:
display(pd.Series(y_pred_logis).value_counts())
display(pd.Series(y_teste).value_counts())


0    80
1    57
dtype: int64

0    70
1    67
Name: ICU, dtype: int64

> Note para os valores ``y_pred_logis`` temos a estimativa de 61 pacientes que precisaram da UTI, entretanto na realidade ``y_teste`` temos que 64 realmente precisaram. Isso significa que nosso modelo classificou 3 pacientes que precisavam como pessoas que não precisavam do serviço, dizemos que estes pacientes receberam um *falso negativo*. 

> **Precisão:** Indica, das classificações corretas, quantas o modelo acertou.

> **Revocação**: Indica, das amostras corretas existentes, quantas o modelo conseguiu classificar corretamente.

## Ajustando ``PATIENT_VISIT_IDENTIFIER``

Dentro da base de dados do Sirio Libanês tem uma coluna de identificação do paciente chamada ``PATIENT_VISIT_IDENTIFIER``, esta coluna serve para indicar **um registro** de um mesmo paciente pode fazer varios exames. Veja os 5 primeiros elementos e os 5 ultimos. Note também que um mesmo paciente tem dados cujo valor é ``NaN``, isso acontece pode pode acontecer do paciente não ter efetuado o exame.

Portanto precisamos ajustar essas informações:

In [28]:
sirio_libanes = pd.read_excel('https://github.com/ConradBitt/BootCamp_DataScience/blob/master/ML%20em%20Saude/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true')
sirio_libanes

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,0.318681,0.898990,NaN,NaN,NaN,NaN,-0.275362,0.736842,NaN,NaN,NaN,NaN,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,-1.0,-0.871658,-0.871658,-0.871658,-0.871658,-1.0,-0.863874,-0.863874,-0.863874,-0.863874,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.414634,-0.414634,-0.414634,-0.414634,-1.0,-0.979069,-0.979069,...,-0.996762,-1.0,-0.243021,-0.338537,-0.213031,-0.317859,0.033779,0.665932,-0.283951,-0.376923,-0.188679,-0.379310,0.035714,0.631579,-0.340206,-0.4875,-0.572650,-0.857143,0.098901,0.797980,-0.076923,0.286486,0.298507,0.272727,0.362319,0.947368,-0.339130,0.325153,0.114504,0.176471,-0.238095,-0.818182,-0.389967,0.4075

Para resolver este problema, uma forma de estimar um valor possível para o ``NaN`` é preencher o valor anterior com o valor atual e o valor posterior também.

No pandas podemos realizar estre processo com o método ``.fillna()`` através do hiperparâmetro ``method`` podemos definir o *back fill* ou *foward fill*.

In [29]:
def preenche_tabela(dados):

  features_categoricas = dados.iloc[:, :13]

  saida = dados.iloc[:,-2:]

  colunas_features_continuas = dados.iloc[:,13:-2].columns
  features_continuas = dados.groupby('PATIENT_VISIT_IDENTIFIER')[colunas_features_continuas].fillna(method='backfill').fillna(method='ffill')
  
  dados_limpos = pd.concat([features_categoricas, features_continuas, saida], axis=1)

  dados_limpos.columns = dados.columns

  return dados_limpos

In [30]:
dados_limpos = preenche_tabela(sirio_libanes)

> Alguns pacientes dão entrada direto na UTI, logo nas primeiras 2 horas. Precisamos retirar esses pacientes porque eles não tem representatividade preditiva, pois se um paciente entrou direto na UTI nas duas primeiras horas os exames realizados provavelmente foram feitos dentro da UTI.

> Portanto, vamos verificar quantos são os pacientes que entraram na UTI nas duas primeiras horas:

In [31]:
dados_limpos.query('WINDOW == "0-2" and ICU == 1')

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
5,1,1,90th,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,-0.210526,-0.210526,-0.210526,-0.210526,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.219512,-0.219512,-0.219512,-0.219512,-1.0,-0.967556,-0.967556,...,-0.978029,-1.0,-0.283951,-0.046154,1.886792e-01,0.830508,-0.107143,1.000000,-0.283951,-0.046154,1.886792e-01,0.862069,-0.107143,1.000000,-0.072165,0.1500,0.264957,1.000000,0.318681,1.000000,-0.504274,-0.329730,-0.059701,0.636364,-0.275362,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
85,17,0,40th,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.365854,-0.365854,-0.365854,-0.365854,-1.0,-0.365854,-0.365854,-0.365854,-0.365854,-1.0,-0.959184,-0.959184,...,-0.937864,-1.0,0.135802,0.153846,-5.471698e-01,-0.525424,-0.071429,0.947368,0.135802,0.153846,-5.471698e-01,-0.517241,-0.071429,0.947368,0.278351,0.3125,-0.401709,-0.428571,0.340659,0.979798,-0.213675,-0.189189,-0.641791,-0.575758,-0.246377,0.947368,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
135,27,1,60th,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-1.0,-0.679144,-0.679144,-0.679144,-0.679144,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.219512,-0.219512,-0.219512,-0.219512,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.966510,-0.966510,...,-0.967082,-1.0,0.209877,0.030769,-5.660377e-02,-0.525424,0.214286,0.684211,0.209877,0.030769,-5.660377e-02,-0.517241,0.214286,0.684211,0.340206,0.2125,0.042735,-0.428571,0.516484,0.878788,-0.162393,-0.275676,-0.253731,-0.575758,-0.014493,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
205,41,1,70th,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.738220,-0.738220,-0.738220,-0.738220,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.170732,-0.170732,-0.170732,-0.170732,-1.0,-0.976975,-0.976975,...,-0.524265,-1.0,-0.104056,-0.059341,-8.328841e-01,-0.118644,-0.214286,0.834586,-0.061728,-0.138462,-8.490566e-01,-0.103448,-0.142857,0.736842,-0.154639,-0.1750,-0.675214,0.000000,-0.010989,0

In [32]:
identificador_dos_pacientes_a_remover = dados_limpos.query('WINDOW == "0-2" and ICU == 1')['PATIENT_VISIT_IDENTIFIER'].values
identificador_dos_pacientes_a_remover

array([  1,  17,  27,  41,  47,  48,  54,  62,  85,  90, 104, 106, 109,
       117, 144, 176, 202, 206, 226, 233, 239, 261, 270, 282, 286, 306,
       315, 329, 359, 362, 366, 379])

In [33]:
dados_limpos = dados_limpos.query(f'PATIENT_VISIT_IDENTIFIER not in "{identificador_dos_pacientes_a_remover.tolist}"')

In [34]:
dados_limpos.describe()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,...,DIMER_MIN,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
count,1925.000000,1925.000000,1925.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,...,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000
mean,192.000000,0.467532,0.368831,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,0.213021,0.158333,0.809896,0.516213,0.516213,0.516213,0.516213,-1.0,-0.955636,-0.955636,-0.955636,-0.955636,-1.0,-0.925260,-0.925260,-0.925260,-0.925260,-1.0,-0.306151,-0.306151,-0.306151,-0.306151,-1.0,-0.306658,-0.306658,-0.306658,-0.306658,-1.0,-0.946290,-0.946290,-0.946290,...,-0.954853,-0.954853,-1.0,-0.080520,-0.333571,-0.264856,-0.443451,0.066566,0.747798,-0.084355,-0.338761,-0.268233,-0.437976,0.063237,0.752317,-0.012805,-0.189747,-0.246780,-0.462931,0.343399,0.837630,-0.246878,-0.420799,-0.306649,-0.355907,-0.013680,0.810841,-0.788196,-0.769867,-0.795555,-0.758319,-0.811824,-0.910237,-0.817464,-0.758728,-0.848568,-0.771306,-0.812740,-0.910107,0.267532
std,111.168431,0.499074,0.482613,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,0.409549,0.365148,0.392485,0.235482,0.235482,0.235482,0.235482,0.0,0.180870,0.180870,0.180870,0.180870,0.0,0.178368,0.178368,0.178368,0.178368,0.0,0.107166,0.107166,0.107166,0.107166,0.0,0.121658,0.121658,0.121658,0.121658,0.0,0.085186,0.085186,0.085186,...,0.121905,0.121905,0.0,0.251287,0.269114,0.242959,0.219027,0.244714,0.124261,0.257442,0.272648,0.248633,0.229689,0.252491,0.121534,0.276894,0.270700,0.264318,0.267036,0.211571,0.244061,0.252048,0.270569,0.278271,0.373856,0.267443,0.133631,0.335760,0.377467,0.335971,0.433037,0.296595,0.255396,0.301522,0.389168,0.246814,0.400915,0.295435,0.256035,0.442787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

## Preparando dados

Agora que nós temos os pacientes, precisamos seprar o dados relevantes de cada um, para isso vamos criar uma fução:

In [35]:
def prepare_window(rows):

  # se ao longo da coluna ['ICU'] tiver algum valor:
  if (np.any(rows['ICU'])):

    rows.loc[rows['WINDOW'] == '0-2', 'ICU'] = 1
  return rows.loc[rows['WINDOW'] == '0-2']

In [36]:
dados_limpos = dados_limpos.groupby('PATIENT_VISIT_IDENTIFIER').apply(prepare_window)

In [37]:
 dados_limpos.head()

,,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
1,5,1,1,90th,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,-0.210526,-0.210526,-0.210526,-0.210526,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.219512,-0.219512,-0.219512,-0.219512,-1.0,-0.967556,-0.967556,...,-0.978029,-1.0,-0.283951,-0.046154,0.188679,0.830508,-0.107143,1.000000,-0.283951,-0.046154,0.188679,0.862069,-0.107143,1.000000,-0.072165,0.1500,0.264957,1.000000,0.318681,1.000000,-0.504274,-0.329730,-0.059701,0.636364,-0.275362,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
2,10,2,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.978029,-1.0,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,0.959596,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
3,15,3,0,40th,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-0.263158,-0.263158,-0.263158,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.972789,-0.972789,...,-0.978029,-1.0,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,0.878788,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,-1.0000

## Rodando novamente o modelo:


In [38]:
colunas_quantitativas = dados_limpos.describe().columns

sirio_libanes_quantitativos = dados_limpos[colunas_quantitativas].dropna()
sirio_libanes_quantitativos.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 384 entries, (0, 0) to (384, 1920)
Columns: 229 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: float64(225), int64(4)
memory usage: 704.6 KB


In [39]:
y = sirio_libanes_quantitativos['ICU']
x = sirio_libanes_quantitativos.drop('ICU', axis=1)

In [40]:
import numpy as np

seed = np.random.seed(120925)

epocas2 = {}
for epoca in range(10):
  # Instanciando modelos
  modelo_dummy = DummyClassifier()
  modelo_regressao_logistica = LogisticRegression(solver='newton-cg')

  # Separação dos dados
  x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, random_state=seed)

  # Ajuste dos modelos
  modelo_dummy.fit(x_treino,y_treino)
  modelo_regressao_logistica.fit(x_treino,y_treino)

  # Predição dos modelos
  y_pred_dummy = modelo_dummy.predict(x_teste.to_numpy())
  y_pred_logis = modelo_regressao_logistica.predict(x_teste.to_numpy())

  # Métricas de qualidade
  acc_dummy = accuracy_score(y_teste, y_pred_dummy)
  acc_logis = accuracy_score(y_teste, y_pred_logis)

  # Exibindo méétricas
  epocas2[epoca] = (acc_dummy, acc_logis)

epoca2 = pd.DataFrame(epocas2, index=['Dummy','Logistic']).T.describe()

for chave, valor in epocas.items():
  print(f'Epoca {chave} - '+'\033[34m'+f'Acurácia Dummy: {valor[0].round(2)*100:.2f}% | Acurácia Logistic: {valor[1].round(2)*100:.2f}% '+'\033[0;0m')

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarn

Epoca 0 - Acurácia Dummy: 47.00% | Acurácia Logistic: 80.00% 
Epoca 1 - Acurácia Dummy: 47.00% | Acurácia Logistic: 90.00% 
Epoca 2 - Acurácia Dummy: 54.00% | Acurácia Logistic: 85.00% 
Epoca 3 - Acurácia Dummy: 50.00% | Acurácia Logistic: 88.00% 
Epoca 4 - Acurácia Dummy: 52.00% | Acurácia Logistic: 89.00% 
Epoca 5 - Acurácia Dummy: 45.00% | Acurácia Logistic: 85.00% 
Epoca 6 - Acurácia Dummy: 48.00% | Acurácia Logistic: 90.00% 
Epoca 7 - Acurácia Dummy: 48.00% | Acurácia Logistic: 91.00% 
Epoca 8 - Acurácia Dummy: 49.00% | Acurácia Logistic: 88.00% 
Epoca 9 - Acurácia Dummy: 42.00% | Acurácia Logistic: 85.00% 


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [41]:
pd.DataFrame(epocas, index=['Dummy','Logistic']).T.describe()

,Dummy,Logistic
count,10.000000,10.000000
mean,0.481752,0.870803
std,0.034581,0.032288
min,0.416058,0.802920
25%,0.474453,0.848540
50%,0.481752,0.883212
75%,0.494526,0.895985
max,0.540146,0.905109


Note que a acurácia mudou bastante. Antes a média da acurácia das épocas era entre $80\%$ e $90\%$, agora criando uma descrição melhor da realidade a acurácia média é de $72\%$. 

> Um ponto importante de olhar os percentis da acurácia do modelo logistico éé identificar que a menor precisão dele é de $68\%$. Ou seja, se comparado com a melhor das chances do Dummy ($54\%$), o modelo logistico é 14 pontos percentuais melhor do que um mero chute. 

> Outro detalhe é para 10 épocas, a acurácia máxima do modelo logistico foi de $79\%$

Comparando o resultado dos conjuntos de épocas, podemos verificar que ocorreu uma reduçãão na precisão. Isso acontece porque ao refinar a base de dados fornecemos dados mais realistas logo modelo também ficou mais realista:

In [53]:
epoca2 - epoca1

,Dummy,Logistic
count,0.000000,0.000000
mean,-0.025623,-0.123502
std,0.003578,0.015864
min,-0.010873,-0.137318
25%,-0.035128,-0.139941
50%,-0.027600,-0.130589
75%,-0.024502,-0.111333
max,-0.005779,-0.106144


In [52]:
epoca2 - epoca1

,Dummy,Logistic
count,0.000000,0.000000
mean,-0.025623,-0.123502
std,0.003578,0.015864
min,-0.010873,-0.137318
25%,-0.035128,-0.139941
50%,-0.027600,-0.130589
75%,-0.024502,-0.111333
max,-0.005779,-0.106144


# Conclusão

> Utilizamos a base de dados disponível no Kaggle. Lá existem divérsas bases de dados compartilhaveis, respeitando a Lei Geral de Proteção de Dados.

> Foi tirada uma conclusão errada a respeito do achismo de "**Cada linha é um paciente**". Entretanto, cada linha é um registro de uma bateria de exames. Além disso um paciente pode ir direto pra UTI, o que torna váias informações não tão importantes para o modelo, afinal não faz sentido tentar prever a entrada de um paciente numa UTI sendo que ele já esta na UTI... Por isso é importante ficar atento: "**Em geral dados de bancos de dados relacionais e excel são armazenados registros de ações e não uma única informação sobre o paciente.**"

> Diferenças entre classificação e regressão e porque uma regressão logística pode ser utilizada para classificação binária. E testes referênte a este modelo. Qual a diferença de um modelo estatistico e um mero chute com o Dummy que estima um valor mais frequênte ou outra abordagem mais ingênua. Além disso foi falado a respeito do problema do viés em utilizar dados de treino para fazer uma predição.

> Foi visto também a qualidade de uma predição utilizando métricas, mais precisamente a acurácia. Foi citado também alguns problemas entre precisão e revocação.

> Outro ponto importante que foi abordado é a reprodutibilidade dos resultados com o ``np.random.seed()`` e o porquê não é uma boa prática utilizar o número $42$ ou $0$ como semente.

